# Оглавление

**Table of contents**<a id='toc0_'></a>    
- [КОД-1. Подготовка](#toc1_)    
    - [Импортируем библиотеку importlib](#toc1_1_1_)    
    - [Импортируем модуль 'tiktoken'](#toc1_1_2_)    
    - [Импорт необходимых модулей](#toc1_1_3_)    
    - [Для начала, создадим переменные, в которых будут находится тексты, которые мы будем подавать в system и user:](#toc1_1_4_)    
    - [System: От имени кого нужно работать с текстом](#toc1_1_5_)    
    - [base_info](#toc1_1_6_)    
- [Задаем переменные: question и assist](#toc2_)    
- [КОД-2. Проверка количества токенов и исправление ошибок](#toc3_)    
  - [Проверяем количество токенов](#toc3_1_)    
  - [Проверка правописания](#toc3_2_)    
      - [Вариант 2. Новичок](#toc3_2_1_1_)    
      - [Вариант 3 / Опытный программист](#toc3_2_1_2_)    
      - [Вариант 1-2 / temperature=1 / без assistant](#toc3_2_1_3_)    
      - [Вариант 2-2 / temperature=1 / Наинающий программист](#toc3_2_1_4_)    
      - [Вариант 3-2 / temperature=1 / Опытный программист](#toc3_2_1_5_)    
- [Код с различными параметрами assistant и temperature оборачиваем в функции](#toc4_)    
- [Работа функций](#toc5_)    
    - [Assistant = NO / Temperature = 0.5](#toc5_1_1_)    
    - [Assistant = Начинающий специалист ... / Temperature = 0.5](#toc5_1_2_)    
    - [Assistant = Опытный инженер-программист ... / Temperature = 0.5](#toc5_1_3_)    
    - [Assistant = NO / Temperature = 1](#toc5_1_4_)    
    - [Assistant = Начинающий специалист ... / Temperature = 1](#toc5_1_5_)    
    - [Assistant = Опытный инженер-программист ... / Temperature = 1](#toc5_1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[КОД-1. Подготовка](#toc0_)

### <a id='toc1_1_1_'></a>[Импортируем библиотеку importlib](#toc0_)

In [35]:
import importlib

# Пытаемся импортировать модуль 'openai'
try:
    importlib.import_module('openai')
    print('Модуль "openai" установлен ранее.')
# Если импорт не удался, устанавливаем модуль 'openai' с помощью pip
except ImportError:
    !pip install openai
    print('Модуль "openai" установлен.')

Модуль "openai" установлен ранее.


### <a id='toc1_1_2_'></a>[Импортируем модуль 'tiktoken'](#toc0_)

In [36]:
# Пытаемся импортировать модуль 'tiktoken'
try:
    importlib.import_module('tiktoken')
    print('Модуль "tiktoken" установлен ранее.')
# Если импорт не удался, устанавливаем модуль 'tiktoken' с помощью pip
except ImportError:
    !pip install tiktoken
    print('Модуль "tiktoken" установлен.')

Модуль "tiktoken" установлен ранее.


### <a id='toc1_1_3_'></a>[Импорт необходимых модулей](#toc0_)

In [37]:
import getpass
import openai
import os

import importlib
import tiktoken

In [38]:
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

### <a id='toc1_1_4_'></a>[Для начала, создадим переменные, в которых будут находится тексты, которые мы будем подавать в system и user:](#toc0_)

### <a id='toc1_1_5_'></a>[System: От имени кого нужно работать с текстом](#toc0_)

In [39]:
# это пока лучшая инструкция

system = '''Ты являешься опытным и добросовестным лингвистом, специализирующимся на коррекции резюме соискателей, работающих в области разработки искусственного интеллекта на русском языке.
У тебя имеется подробная инструкция, которая описывает, как правильно выполнять коррекцию русского текста. Тебе предоставят текст, и в случае необходимости, ты будешь вносить исправления, соблюдая данную инструкцию.
Ты стремишся максимально точно исправлять текст в соответствии с инструкцией, не упоминая при этом саму инструкцию и примеры из неё. Твоей целью является построение грамматически верных предложений,
придание тексту смысла, исправление всех выявленных ошибок и выстраивая красивую русскую речь. Поэтому, когда тебе будет предоставлен запрос, наподобие "Исправь поле резюме 'Обо мне': 'Инженер по обучению дашинномт опытом работы в с биоинформатики сфере'",
после твоей коррекции, текст должен выглядеть так: "Инженер по машинному обучению с опытом работы в сфере биоинформатики". Не требуется повторять в твоём ответе текст из запроса пользователя, а следует просто предоставить
исправленную версию текста из резюме соискателя.'''

### <a id='toc1_1_6_'></a>[base_info](#toc0_)

In [40]:
base_info = '''Понимание задачи: Перед началом работы удостоверьтесь, что вы понимаете контекст и цель текста. Это поможет вам правильно интерпретировать предложения и выявить ошибки.
Проверка орфографии: Внимательно проверьте орфографию слов. Обратите внимание на правильность написания букв, прописных и строчных букв, использование буквы ё и другие орфографические правила.
Проверка пунктуации: Убедитесь, что пунктуация в тексте соответствует правилам. Проверьте запятые, точки, тире, кавычки и другие знаки препинания.
Согласование времен и лиц: Проверьте согласование времен и лиц в предложениях. Убедитесь, что временные формы глаголов и лица согласованы.
Проверка синтаксиса: Проверьте структуру предложений и их логическую связь. Устраните ошибки, связанные с неправильным порядком слов или непонятной структурой предложений.
Проверка согласования: Удостоверьтесь, что существительные, прилагательные и глаголы согласованы в роде, числе и падеже.
Использование словарей и справочников: Пользуйтесь словарями, доступными вам, стилистическими справочниками и грамматическими ресурсами для подтверждения правильности исправлений.
Связность и структура: Обратите внимание на связность текста и его общую структуру. Убедитесь, что переходы между абзацами и идеями логичны.
Контроль за повторами: Проверьте текст на повторяющиеся слова, фразы или идеи и устраните их или замените синонимами, если это необходимо.
Проверка стиля и тона: Учтите стиль и тональность текста в соответствии с его назначением. В случае необходимости, предложите исправления, чтобы улучшить стиль и язык текста.
Раздели текст на абзацы по смыслу. Финальное чтение: После всех исправлений выполните финальное чтение текста для проверки, убедитесь, что все ошибки устранены и текст готов к публикации.'''

# <a id='toc2_'></a>[Задаем переменные: question и assist](#toc0_)

In [41]:
question = '''В программирование я с 1995 года. От создания скриптов до создания продающих сайтов и мобильных приложений. 
Последние 4 года занимаюсь программированием на языке python. Специализация - искусственный интеллект в компьютерном зрении и чат-боты на базе искусственного интеллекта и ChatGPT. 
Учавствовал в разработке таких проектов: 
- Определение груза поднимаемого башенным краном.
- Опредление человека с оружием и сигналирование охране.
- Разработка сайта для создания резюме сотрудников с использованием ChatGPT
'''

In [42]:
assist1 = '''Начинающий специалист в области Data Science. 
Умею решать задачи глубокого машинного обучения, компьютерного зрения, компьютерного перевода, а также имею опыт работы с системами распознавания образов. 
Постоянно участвую в соревнованиях - хакатонах, занимая призовые места.
'''

In [43]:
assist2 = '''Опытный инженер-программист с опытом разработки ПО для автоматических систем управления. 
Владею фронтендом и бэкендом на базе систем SCADA, а также специализируюсь на лабораторных испытательных комплексах. 
Стремлюсь профессионально развиваться в области искусственного интеллекта, чтобы помочь внедрять его в повседневные задачи, бизнес-процессы и АСУ-приложения. 
Меня интересуют проекты, которые представляют реальную пользу и развитие человечества.
'''

# <a id='toc3_'></a>[КОД-2. Проверка количества токенов и исправление ошибок](#toc0_)

## <a id='toc3_1_'></a>[Проверяем количество токенов](#toc0_)

In [44]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Возвращает количество токенов, используемых списком сообщений."""
    try:
        encoding = tiktoken.encoding_for_model(model) # Пытаемся получить кодировку для выбранной модели
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base") # если не получается, используем кодировку "cl100k_base"
    if model == "gpt-3.5-turbo-0301" or "gpt-3.5-turbo-0613" or "gpt-3.5-turbo-16k" or "gpt-3.5-turbo":
        num_tokens = 0 # начальное значение счетчика токенов
        for message in messages: # Проходимся по каждому сообщению в списке сообщений
            num_tokens += 4  # каждое сообщение следует за <im_start>{role/name}\n{content}<im_end>\n, что равно 4 токенам
            for key, value in message.items(): # итерация по элементам сообщения (роль, имя, контент)
                num_tokens += len(encoding.encode(value)) # подсчет токенов в каждом элементе
                if key == "name":  # если присутствует имя, роль опускается
                    num_tokens += -1  # роль всегда требуется и всегда занимает 1 токен, так что мы вычитаем его, если имя присутствует
        num_tokens += 2  # каждый ответ начинается с <im_start>assistant, что добавляет еще 2 токена
        return num_tokens # возвращаем общее количество токенов
    else:
      # Если выбранная модель не поддерживается, генерируем исключение
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}. # вызываем ошибку, если функция не реализована для конкретной модели""")


## <a id='toc3_2_'></a>[Проверка правописания](#toc0_)

In [45]:
# множественный запрос разбитый на отдельные предложения дает ответы без текста с запросами пользователя
question = '''Исправь текст: ''' + question + '''не пиши "исправленный текст"'''

In [46]:
messages = [
    {"role": "system", "content": system},  # Сообщение от системы
    
    {"role": "user", "content": base_info + f"{question}"}  # Сообщение от пользователя, содержащее информацию и вопрос
    # {"role": "user", "content": question}  # Эта строка закомментирована, возможно, она была использована ранее
    ]
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",  # Выбранная модель для генерации текста
    messages=messages,  # Список сообщений, на основе которых будет сгенерирован текст
    temperature=0.0,  # Параметр, отвечающий за "творческий потенциал" модели
    max_tokens=1000  # Максимальное количество токенов в сгенерированном тексте
    )
message = completion['choices'][0]['message']['content']  # Полученный ответ от модели
print("\nТекст скорректированный ChatGPT:\n\n")  # Вывод заголовка
print(message)  # Вывод сгенерированного текста



Текст скорректированный ChatGPT:


Я занимаюсь программированием с 1995 года, начиная от создания скриптов и заканчивая разработкой продающих сайтов и мобильных приложений. В последние 4 года я специализируюсь на программировании на языке Python, с фокусом на искусственный интеллект в компьютерном зрении и создание чат-ботов на базе искусственного интеллекта и ChatGPT. Моими достижениями в этой области являются:
- Разработка системы для определения груза, поднимаемого башенным краном.
- Создание системы для обнаружения человека с оружием и автоматического сигналирования охране.
- Разработка сайта для создания резюме сотрудников с использованием ChatGPT.


#### <a id='toc3_2_1_1_'></a>[Вариант 2. Новичок](#toc0_)

#### <a id='toc3_2_1_2_'></a>[Вариант 3 / Опытный программист](#toc0_)

---

#### <a id='toc3_2_1_3_'></a>[Вариант 1-2 / temperature=1 / без assistant](#toc0_)

#### <a id='toc3_2_1_4_'></a>[Вариант 2-2 / temperature=1 / Наинающий программист](#toc0_)

#### <a id='toc3_2_1_5_'></a>[Вариант 3-2 / temperature=1 / Опытный программист](#toc0_)

---
---

# <a id='toc4_'></a>[Код с различными параметрами assistant и temperature оборачиваем в функции](#toc0_)

In [47]:
# Assistant = NO / Temperature = 0.5

def gptdialog_1_t_0_a_no(userquest):
    result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": system},                  # От имени кого
            {"role": "user", "content": base_info+ f"{userquest}"}   # Запрос
        ],
    # max_tokens=max_tokens, # Максимальное количество токенов в ответе. Если токены кончатся, ответ оборвется незаконченым
    temperature=0.5, # Значение от 0 до 2, рекомендуется от 0 до 1
    # n=1 # Количество ответов от GPT 
    )

    message = result['choices'][0]['message']['content']

    import requests
    import textwrap

    #url = 'https://www.google.com/'
    #response = requests.get(url)

    # Wrap the response text to a width of 80 characters
    wrapped_text = textwrap.fill(message, width=80)

    print(wrapped_text)

In [48]:
# Assistant = Начинающий специалист ... / Temperature = 0.5

def gptdialog_2_t_05_a_1(userquest):
    result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": system},                  # От имени кого
            {"role": "assistant", "content": assist1},
            {"role": "user", "content": base_info+ f"{userquest}"}   # Запрос
        ],
    # max_tokens=max_tokens, # Максимальное количество токенов в ответе. Если токены кончатся, ответ оборвется незаконченым
    temperature=0.5, # Значение от 0 до 2, рекомендуется от 0 до 1
    # n=1 # Количество ответов от GPT 
    )

    message = result['choices'][0]['message']['content']

    import requests
    import textwrap

    #url = 'https://www.google.com/'
    #response = requests.get(url)

    # Wrap the response text to a width of 80 characters
    wrapped_text = textwrap.fill(message, width=80)

    print(wrapped_text)

In [49]:
# Assistant = Опытный инженер-программист ... / Temperature = 0.5

def gptdialog_3_t_05_a_2(userquest):
    result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": system},                  # От имени кого
            {"role": "assistant", "content": assist2},
            {"role": "user", "content": base_info+ f"{userquest}"}   # Запрос
        ],
    # max_tokens=max_tokens, # Максимальное количество токенов в ответе. Если токены кончатся, ответ оборвется незаконченым
    temperature=0.5, # Значение от 0 до 2, рекомендуется от 0 до 1
    # n=1 # Количество ответов от GPT 
    )

    message = result['choices'][0]['message']['content']

    import requests
    import textwrap

    #url = 'https://www.google.com/'
    #response = requests.get(url)

    # Wrap the response text to a width of 80 characters
    wrapped_text = textwrap.fill(message, width=80)

    print(wrapped_text)

In [50]:
# Assistant = NO / Temperature = 1

def gptdialog_4_t_1_a_no(userquest):
    result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": system},                  # От имени кого
            {"role": "user", "content": base_info+ f"{userquest}"}   # Запрос
        ],
    # max_tokens=max_tokens, # Максимальное количество токенов в ответе. Если токены кончатся, ответ оборвется незаконченым
    temperature=1, # Значение от 0 до 2, рекомендуется от 0 до 1
    # n=1 # Количество ответов от GPT 
    )

    message = result['choices'][0]['message']['content']

    import requests
    import textwrap

    #url = 'https://www.google.com/'
    #response = requests.get(url)

    # Wrap the response text to a width of 80 characters
    wrapped_text = textwrap.fill(message, width=80)

    print(wrapped_text)

In [51]:
# Assistant = Начинающий специалист ... / Temperature = 1

def gptdialog_5_t_1_a_1(userquest):
    result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": system},                  # От имени кого
            {"role": "assistant", "content": assist1},
            {"role": "user", "content": base_info+ f"{userquest}"}   # Запрос
        ],
    # max_tokens=max_tokens, # Максимальное количество токенов в ответе. Если токены кончатся, ответ оборвется незаконченым
    temperature=1, # Значение от 0 до 2, рекомендуется от 0 до 1
    # n=1 # Количество ответов от GPT 
    )

    message = result['choices'][0]['message']['content']

    import requests
    import textwrap

    #url = 'https://www.google.com/'
    #response = requests.get(url)

    # Wrap the response text to a width of 80 characters
    wrapped_text = textwrap.fill(message, width=80)

    print(wrapped_text)

In [52]:
# Assistant = Опытный инженер-программист ... / Temperature = 1

def gptdialog_6_t_1_a_2(userquest):
    result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": system},                  # От имени кого
            {"role": "assistant", "content": assist2},
            {"role": "user", "content": base_info+ f"{userquest}"}   # Запрос
        ],
    # max_tokens=max_tokens, # Максимальное количество токенов в ответе. Если токены кончатся, ответ оборвется незаконченым
    temperature=1, # Значение от 0 до 2, рекомендуется от 0 до 1
    # n=1 # Количество ответов от GPT 
    )

    message = result['choices'][0]['message']['content']

    import requests
    import textwrap

    #url = 'https://www.google.com/'
    #response = requests.get(url)

    # Wrap the response text to a width of 80 characters
    wrapped_text = textwrap.fill(message, width=80)

    print(wrapped_text)

---
---

# <a id='toc5_'></a>[Работа функций](#toc0_)

In [54]:
question

'Исправь текст: В программирование я с 1995 года. От создания скриптов до создания продающих сайтов и мобильных приложений. \nПоследние 4 года занимаюсь программированием на языке python. Специализация - искусственный интеллект в компьютерном зрении и чат-боты на базе искусственного интеллекта и ChatGPT. \nУчавствовал в разработке таких проектов: \n- Определение груза поднимаемого башенным краном.\n- Опредление человека с оружием и сигналирование охране.\n- Разработка сайта для создания резюме сотрудников с использованием ChatGPT\nне пиши "исправленный текст"'

# Цели теста:
### 1. Внедрение роли ASSISTANT
### 2. Проверка его работы при параметре TEMPERATURE = 0.5 и 1

### <a id='toc5_1_1_'></a>[Assistant = NO / Temperature = 0.5](#toc0_)

In [61]:
gptdialog_1_t_0_a_no(question)

Я занимаюсь программированием с 1995 года, включая создание скриптов, продающих
сайтов и мобильных приложений. В последние 4 года я специализируюсь на языке
Python и работаю в области искусственного интеллекта в компьютерном зрении и
создании чат-ботов на базе искусственного интеллекта и ChatGPT. Моими
достижениями является участие в разработке следующих проектов:  - Определение
груза, поднимаемого башенным краном. - Определение человека с оружием и
сигнализация охране. - Разработка сайта для создания резюме сотрудников с
использованием ChatGPT.


### <a id='toc5_1_2_'></a>[Assistant = Начинающий специалист ... / Temperature = 0.5](#toc0_)

In [56]:
gptdialog_2_t_05_a_1(question)

Я занимаюсь программированием с 1995 года, от создания скриптов до разработки
продающих сайтов и мобильных приложений. Последние 4 года я специализируюсь на
языке Python и работе с искусственным интеллектом в компьютерном зрении и
создании чат-ботов на основе искусственного интеллекта и ChatGPT. Мои проекты
включают: - Определение груза, поднимаемого башенным краном. - Обнаружение
человека с оружием и оповещение службы безопасности. - Разработка сайта для
создания резюме сотрудников с использованием ChatGPT.


### <a id='toc5_1_3_'></a>[Assistant = Опытный инженер-программист ... / Temperature = 0.5](#toc0_)

In [57]:
gptdialog_3_t_05_a_2(question)

Я занимаюсь программированием с 1995 года. Мой опыт включает создание скриптов,
разработку продающих сайтов и мобильных приложений.  В течение последних 4 лет я
специализируюсь на программировании на языке Python. Моя область специализации -
искусственный интеллект в компьютерном зрении и создание чат-ботов на основе
искусственного интеллекта и ChatGPT.  Я принимал участие в разработке следующих
проектов:  - Определение груза, поднимаемого башенным краном. - Определение
человека с оружием и передача сигнала охране. - Разработка сайта для создания
резюме сотрудников с использованием ChatGPT.


### <a id='toc5_1_4_'></a>[Assistant = NO / Temperature = 1](#toc0_)

In [58]:
gptdialog_4_t_1_a_no(question)

Я занимаюсь программированием с 1995 года, начиная с создания скриптов и
заканчивая разработкой продуктивных веб-сайтов и мобильных приложений. В
последние 4 года моя специализация - искусственный интеллект в области
компьютерного зрения и создание чат-ботов на основе искусственного интеллекта,
таких как ChatGPT. Мои проекты включают:  - Определение груза, поднимаемого
башенным краном. - Обнаружение лица с оружием и автоматическая отправка сигнала
охране. - Разработка веб-сайта для создания резюме с использованием ChatGPT.


### <a id='toc5_1_5_'></a>[Assistant = Начинающий специалист ... / Temperature = 1](#toc0_)

In [59]:
gptdialog_5_t_1_a_1(question)

Я занимаюсь программированием с 1995 года, начиная от создания скриптов до
разработки продающих сайтов и мобильных приложений. В последние 4 года я
специализируюсь на языке Python, в основном работая с искусственным интеллектом
в компьютерном зрении и создании чат-ботов на базе искусственного интеллекта и
ChatGPT. У меня есть опыт работы над следующими проектами:  - Определение груза,
поднимаемого башенным краном. - Определение человека с оружием и оповещение
охраны. - Разработка сайта для создания резюме сотрудников с использованием
ChatGPT.


### <a id='toc5_1_6_'></a>[Assistant = Опытный инженер-программист ... / Temperature = 1](#toc0_)

In [60]:
gptdialog_6_t_1_a_2(question)

Я работаю в сфере программирования с 1995 года. Мой опыт включает создание
скриптов, разработку продающих сайтов и мобильных приложений.  За последние 4
года я специализировался на языке Python и работе с искусственным интеллектом в
компьютерном зрении и создании чат-ботов на основе искусственного интеллекта и
ChatGPT.  Моя работа включала участие в таких проектах, как: - Разработка
системы для определения груза, поднимаемого башенным краном. - Разработка
системы для определения человека с оружием и передачи сигнала охране. - Создание
сайта для создания резюме сотрудников с использованием ChatGPT.


![Alt text](image-1.png)